In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df=pd.read_csv("/content/train.csv")
df=df.head(100000)

In [ ]:
pip install nltk

In [ ]:
df.isnull()

,id,qid1,qid2,question1,question2,is_duplicate
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
99995,False,False,False,False,False,False
99996,False,False,False,False,False,False
99997,False,False,False,False,False,False
99998,False,False,False,False,False,False


In [ ]:
missing_values = df.isna().sum()

print(missing_values)

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


Thus no need to handle missing values

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            100000 non-null  int64 
 1   qid1          100000 non-null  int64 
 2   qid2          100000 non-null  int64 
 3   question1     100000 non-null  object
 4   question2     100000 non-null  object
 5   is_duplicate  100000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.6+ MB


#ADDING CUSTOM FEATURES

**csc_min=#common stopwords/min of stopwords in both questions and also csc_max**

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


def csc_min(data):
    try:
        # nltk.download('stopwords')
        # nltk.download('punkt')
        # Get the list of stopwords for English
        stop_words = set(stopwords.words('english'))

        # Tokenize the texts into words
        words1 = nltk.word_tokenize(data["question1"])
        words2 = nltk.word_tokenize(data["question2"])

        # Convert the words to lowercase
        words1 = [str(word.lower()) for word in words1]
        words2 = [str(word.lower()) for word in words2]

        # Find the common stopwords
        common_stopwords = set(words1) & set(words2)

        # Count the number of common stopwords
        num_common_stopwords = len(common_stopwords.intersection(stop_words))
    except (TypeError, KeyError):
        print(data)
        # Handle the case where the data is not of the expected type or the expected keys are missing
        return 0

    num_stopwords1 = sum(word in stop_words for word in words1)
    num_stopwords2 = sum(word in stop_words for word in words2)
    min_len=min(num_stopwords1,num_stopwords2)

    return num_common_stopwords/min_len if min_len>0 else num_common_stopwords

In [ ]:
df["csc_min"]=df.apply(csc_min,axis=1)

In [ ]:
def csc_max(data):
    try:
        # nltk.download('stopwords')
        # nltk.download('punkt')
        # Get the list of stopwords for English
        stop_words = set(stopwords.words('english'))

        # Tokenize the texts into words
        words1 = nltk.word_tokenize(data["question1"])
        words2 = nltk.word_tokenize(data["question2"])

        # Convert the words to lowercase
        words1 = [str(word.lower()) for word in words1]
        words2 = [str(word.lower()) for word in words2]

        # Find the common stopwords
        common_stopwords = set(words1) & set(words2)

        # Count the number of common stopwords
        num_common_stopwords = len(common_stopwords.intersection(stop_words))
    except (TypeError, KeyError):
        print(data)
        # Handle the case where the data is not of the expected type or the expected keys are missing
        return 0

    num_stopwords1 = sum(word in stop_words for word in words1)
    num_stopwords2 = sum(word in stop_words for word in words2)
    max_len=max(num_stopwords1,num_stopwords2)

    return num_common_stopwords/max_len if max_len>0 else num_common_stopwords

In [ ]:
df["csc_max"]=df.apply(csc_max,axis=1)

**CTC-min= ratio of number of common tokens to number of tokens in smaller question CTC-max= ratio of number of common tokens to number of tokens in larger question bold text**

In [ ]:
def ctc_min(data):
      try:
          words1 = nltk.word_tokenize(data["question1"])
          words2 = nltk.word_tokenize(data["question2"])

          # Convert the words to lowercase
          words1 = [str(word.lower()) for word in words1]
          words2 = [str(word.lower()) for word in words2]

          # Find the common words
          common_words = np.intersect1d(words1, words2)
          # Get the count of common words
          num_common_words = len(common_words)

          min_len=min(len(words1),len(words2))

          if min_len==0:
            ctc_min=num_common_words/1

          else:
            ctc_min=num_common_words/min_len

          return ctc_min

      except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0

In [ ]:
df['ctc_min']=df.apply(ctc_min,axis=1)

In [ ]:
def ctc_max(data):
      try:
          words1 = nltk.word_tokenize(data["question1"])
          words2 = nltk.word_tokenize(data["question2"])

          # Convert the words to lowercase
          words1 = [str(word.lower()) for word in words1]
          words2 = [str(word.lower()) for word in words2]

          # Find the common words
          common_words = np.intersect1d(words1, words2)
          # Get the count of common words
          num_common_words = len(common_words)

          max_len=max(len(words1),len(words2))

          if max_len==0:
            ctc_max=num_common_words/1

          else:
            ctc_max=num_common_words/max_len

          return ctc_max

      except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0

In [ ]:
df['ctc_max']=df.apply(ctc_max,axis=1)

**Adding longest substr ratio feature where it is equal to length of common longest substring divided by length of smaller question**

In [ ]:
def longest_substr(data):
  try:
   words1 = nltk.word_tokenize(data["question1"])
   words2 = nltk.word_tokenize(data["question2"])

    # Convert the words to lowercase
   words1 = [str(word.lower()) for word in words1]
   words2 = [str(word.lower()) for word in words2]

   count=0

   for i in range(len(words1)):
       for j in range(len(words1)):
          new_list=words1[i:j]
          for l in range(len(words2)):
             if words2[l:l+j-i]==new_list and j-i>count:
                count=j-i

   #finding min length's question
   min_len=min(len(words1),len(words2))

   if min_len==0:
      longest_substr_ratio=count

   else:
      longest_substr_ratio=count/min_len

   return count

  except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0




In [ ]:
df['longest_substr_ratio']=df.apply(longest_substr,axis=1)

**Adding feature of mean_length**

In [ ]:
def mean_len(data):
   try:
    words1 = nltk.word_tokenize(data["question1"])
    words2 = nltk.word_tokenize(data["question2"])

      # Convert the words to lowercase
    words1 = [str(word.lower()) for word in words1]
    words2 = [str(word.lower()) for word in words2]
    return ( len(words1) + len(words2)) / 2
   except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0



In [ ]:
df['mean_length']=df.apply(mean_len,axis=1)

**Adding feature of abs_len_diff**

In [ ]:
def abs_len_diff(data):
   import math
   try:
        words1 = nltk.word_tokenize(data["question1"])
        words2 = nltk.word_tokenize(data["question2"])

          # Convert the words to lowercase
        words1 = [str(word.lower()) for word in words1]
        words2 = [str(word.lower()) for word in words2]
        diff=len(words1) - len(words2)
        return abs(diff)

   except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0


In [ ]:
df['abs_len_diff']=df.apply(abs_len_diff,axis=1)

#Extracting Fuzzy Features

**(https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/)**

Adding partial ratio feature

In [ ]:
pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz
def partial_ratio(data):
  try:
      text1 = data["question1"]
      text2 = data["question2"]
      return fuzz.partial_ratio(str(text1).lower(),str(text2).lower())
  except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0



/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
df['partial_ratio']=df.apply(partial_ratio,axis=1)

Adding token sort ratio

In [ ]:
def token_sort_ratio(data):
  try:
      text1 = data["question1"]
      text2 = data["question2"]
      return fuzz.token_sort_ratio(str(text1).lower(),str(text2).lower())
  except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0

In [ ]:
df['token_sort_ratio']=df.apply(token_sort_ratio,axis=1)

Adding token set ratio

In [ ]:
def token_set_ratio(data):
  try:
      text1 = data["question1"]
      text2 = data["question2"]
      return fuzz.token_set_ratio(str(text1).lower(),str(text2).lower())
  except (TypeError, KeyError):
          print(data)
          # Handle the case where the data is not of the expected type or the expected keys are missing
          return 0

In [ ]:
df['token_set_ratio']=df.apply(token_set_ratio,axis=1)

**CWC-min= ratio of number of common words to number of words in smaller question CWC-max= ratio of number of common words to number of words in larger question bold text**

In [ ]:
def cwc_min(data):
      try:
        import re

        stop_words = set(stopwords.words('english'))

        # Define the pattern for punctuation marks
        punctuation_pattern = r'[^\w\s]'

        # Remove punctuation marks using regular expression substitution
        # Example texts
        text1 = data["question1"]
        text2 = data["question2"]
        text1 = re.sub(punctuation_pattern, ' ', text1)
        text2 = re.sub(punctuation_pattern, ' ', text2)
        # Tokenize the texts into words
        words1 = str(text1).lower().split()
        words2 = str(text2).lower().split()

        # Find the common words
        common_words = np.intersect1d(words1, words2)

        print(common_words)
        # Get the count of common words
        num_common_words = len(common_words)

        #BUT ACTUALLY HERE WORDS MEAN TOKENS OR SIMPLE WORDS-STOP WORDS SO WE DO LIKE THIS
        common_stopwords = set(words1) & set(words2)

        print(common_stopwords)
        # Count the number of common stopwords
        num_common_stopwords = len(common_stopwords.intersection(stop_words))

        num_common_words=num_common_words-num_common_stopwords

        #get min number of words' question
        num_stopwords1 = sum(word in stop_words for word in words1)
        num_stopwords2 = sum(word in stop_words for word in words2)

        min_len=min( len(str(text1).lower().split()) - num_stopwords1,
                    len(str(text2).lower().split()) - num_stopwords2)
        if min_len==0:
          cwc_min=num_common_words/1
        else:
          cwc_min=num_common_words/min_len

        return cwc_min
      except (TypeError, KeyError):
        print(data)
        # Handle the case where the data is not of the expected type or the expected keys are missing
        return 0

In [ ]:
df['cwc_min']=df.apply(ctc_min,axis=1)

In [ ]:
def cwc_max(data):
      try:
        import re

        stop_words = set(stopwords.words('english'))

        # Define the pattern for punctuation marks
        punctuation_pattern = r'[^\w\s]'

        # Remove punctuation marks using regular expression substitution
        # Example texts
        text1 = data["question1"]
        text2 = data["question2"]
        text1 = re.sub(punctuation_pattern, ' ', text1)
        text2 = re.sub(punctuation_pattern, ' ', text2)
        # Tokenize the texts into words
        words1 = str(text1).lower().split()
        words2 = str(text2).lower().split()

        # Find the common words
        common_words = np.intersect1d(words1, words2)

        # Get the count of common words
        num_common_words = len(common_words)

        #BUT ACTUALLY HERE WORDS MEAN TOKENS OR SIMPLE WORDS-STOP WORDS SO WE DO LIKE THIS
        common_stopwords = set(words1) & set(words2)

        # Count the number of common stopwords
        num_common_stopwords = len(common_stopwords.intersection(stop_words))

        num_common_words=num_common_words-num_common_stopwords

        #get min number of words' question
        num_stopwords1 = sum(word in stop_words for word in words1)
        num_stopwords2 = sum(word in stop_words for word in words2)

        max_len=max( len(str(text1).lower().split()) - num_stopwords1,
                    len(str(text2).lower().split()) - num_stopwords2)
        if max_len==0:
          cwc_max=num_common_words/1
        else:
          cwc_max=num_common_words/max_len

        return cwc_max
      except (TypeError, KeyError):
        print(data)
        # Handle the case where the data is not of the expected type or the expected keys are missing
        return 0

In [ ]:
df['cwc_max']=df.apply(cwc_max,axis=1)

**Adding feature of last word equal**




In [ ]:
def l_word_equal(data):
      text1 = data["question1"]
      text2 = data["question2"]
      # Tokenize the texts into words
      words1 = str(text1).lower().split()
      words2 = str(text2).lower().split()
      if words1 and words2:
        if words1[-1]==words2[-1]:
          return 1
        else:
          return 0

In [ ]:
df['last_word_eq']=df.apply(l_word_equal,axis=1)

**Adding feature of first word equal**

In [ ]:
def f_word_equal(data):
      text1 = data["question1"]
      text2 = data["question2"]
      # Tokenize the texts into words
      words1 = str(text1).lower().split()
      words2 = str(text2).lower().split()
      if words1 and words2:
        if words1[-1]==words2[-1]:
          return 1
        else:
          return 0

In [ ]:
df['first_word_eq']=df.apply(f_word_equal,axis=1)

**Now let's apply basic preprocessing
We will remove stop words,use stemming,convert ain't to are not and stuff like this and also try to remove HTML tags and will convert all letters to lower case in upcoming cells**   

In [ ]:
def preprocess(q1):
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    from nltk.tokenize import word_tokenize
    from nltk.tokenize import RegexpTokenizer
    import re

    # Set the language for stopwords
    stop_words = set(stopwords.words('english'))

    pattern = r'\[math\]|\w+|\$[\d\.]+'

    # Create the custom tokenizer
    tokenizer = RegexpTokenizer(pattern)
            #Converting mappings to full words
    word_mapping = {
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "I'd": "I would",
    "I'll": "I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what's": "what is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you're": "you are",
    "you've": "you have",
    "\'m":"km",
    "[math]": "Mathematical Expression or formula"
    # Add more mappings as needed
}
    if not isinstance(q1, str):
        q1 = str(q1)
# Replace contractions with their expanded forms in the sentence
    for contraction, expanded_form in word_mapping.items():
        q1 = q1.replace(contraction, expanded_form)

#removing digits
    pattern = r'\d+'

    # Use the sub() function to remove digits
    q1 = re.sub(pattern, '', q1)

    # Tokenize the sentence
    tokens1 = tokenizer.tokenize(q1)

    q2=[]
    for word in  tokens1:
       if word in word_mapping:
          word=word_mapping[word]
       q2.append(word)
    q1=q2

    #Make object of stemmer
    stemmer = PorterStemmer()


    #Apply stemmer ,this also converts words to lower case and also remove stop words
    q1=" ".join([stemmer.stem(word) for word in q1 if word.lower() not in stop_words])
    return q1


In [ ]:
import tensorflow as tf
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False


In [ ]:
df['question1']=df['question1'].apply(preprocess)

In [ ]:
df['question2']=df['question2'].apply(preprocess)

**Basic preprocessing done and now no need to remove html tags**

In [ ]:
df[df["ctc_min"]!=df["cwc_min"]].head()

,id,qid1,qid2,question1,question2,is_duplicate,csc_min,csc_max,ctc_min,ctc_max,longest_substr_ratio,mean_length,abs_len_diff,partial_ratio,token_sort_ratio,token_set_ratio,cwc_min,cwc_max,last_word_eq,first_word_eq


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:

data={
    "question2" : "Why am I mentally very lonely?How can I solve.",
    "question1" : "How can I go to gym?"}
abs_len_diff(data)

5

In [ ]:
df.tail()

,id,qid1,qid2,question1,question2,is_duplicate,csc_min,csc_max,ctc_min,ctc_max,longest_substr_ratio,mean_length,abs_len_diff,partial_ratio,token_sort_ratio,token_set_ratio,cwc_min,cwc_max,last_word_eq,first_word_eq
99995,99995,165922,165923,icon imag footbal,icon imag women,0,0.857143,0.857143,0.818182,0.818182,9,11.0,0,90,85,93,0.818182,0.666667,0,0
99996,99996,165924,165925,green green tea,green tea green,0,0.500000,0.500000,0.666667,0.444444,1,7.5,3,48,77,72,0.666667,0.666667,0,0
99997,99997,165926,165927,would win black panther batman,would win fight black panther batman,1,0.500000,0.200000,0.777778,0.583333,3,10.5,3,64,74,96,0.777778,0.833333,1,1
99998,99998,165928,165929,school better parson risd,good design school colleg par risd,0,0.200000,0.142857,0.272727,0.200000,1,13.0,4,46,51,51,0.272727,0.166667,1,1
99999,99999,165930,165931,human bodi produc carbon dioxid,would inhal carbon dioxid caus human,0,0.000000,0.000000,0.500000,0.400000,2,9.0,2,49,52,65,0.500000,0.500000,0,0


**Apply Word2Vec** *solve using try and except here the problem*

In [ ]:

from gensim.models import Word2Vec
import nltk
from nltk import sent_tokenize
nltk.download('punkt')
input=[]

def make_corpus(data):
 for i in data:
   import re
   punctuation_pattern = r'[^\w\s]'

    # Remove punctuation marks using regular expression substitution
    # Example texts
   i = re.sub(punctuation_pattern, ' ', i)
   i=str(i).lower()
   raw_sent=sent_tokenize(i)
   for sent in raw_sent:
       input.append(sent.split())
make_corpus(df['question1'])
make_corpus(df['question2'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model=Word2Vec(vector_size=100,window=10,min_count=1)
model.build_vocab(input)
model.train(input,total_examples=model.corpus_count,epochs=model.epochs)


(5147155, 5413970)

In [ ]:
import numpy as np

def transform(data):
    import re
    if data is None:
        print("I am empty")
        return [0] * 100

    from nltk.corpus import stopwords
    # Get the list of stopwords for English
    stop_words = set(stopwords.words('english'))

    # Define the pattern for punctuation marks
    punctuation_pattern = r'[^\w\s]'

    # Remove punctuation marks using regular expression substitution
    data = re.sub(punctuation_pattern, ' ', data)

    # Tokenize the texts into words
    words = str(data).lower().split()
    for t in range(len(words)):
        if words[t] in model.wv:
            words[t] = model.wv[words[t]]
        else:
            print(words[t])

    data = words
    len1 = len(data)

    lists = []
    for lst in data:
        lst = lst.astype(float)
        lists.append(lst)

    if len(lists) > 0:
        return np.mean(lists, axis=0).tolist()
    else:
        return [0] * 100



In [ ]:
df['question1']=df['question1'].apply(transform)

In [ ]:
df['question2']=df['question2'].apply(transform)

In [ ]:
len(df['question1'][0])

100

In [ ]:
import pandas as pd

feature_columns_1 = []
feature_columns_2 = []

for i in range(100):
        feature_columns_1.append(df['question1'].apply(lambda x: x[i] if x is not None else 0).rename(f'feature_q1_{i}'))
        feature_columns_2.append(df['question2'].apply(lambda x: x[i] if x is not None else 0).rename(f'feature_q2_{i}'))

df = pd.concat([df] + feature_columns_1, axis=1)
df = pd.concat([df] + feature_columns_2, axis=1)

# df.drop(['question1', 'question2'], axis=1, inplace=True)



In [ ]:
df.drop(['question1', 'question2'], axis=1, inplace=True)

In [ ]:
df.columns

Index(['id', 'qid1', 'qid2', 'is_duplicate', 'csc_min', 'csc_max', 'ctc_min',
       'ctc_max', 'longest_substr_ratio', 'mean_length',
       ...
       'feature_q2_90', 'feature_q2_91', 'feature_q2_92', 'feature_q2_93',
       'feature_q2_94', 'feature_q2_95', 'feature_q2_96', 'feature_q2_97',
       'feature_q2_98', 'feature_q2_99'],
      dtype='object', length=218)

In [ ]:
X = df.drop("is_duplicate", axis=1)
Y = df["is_duplicate"]

**Train Test Split**

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

**Apply Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(max_depth=2,n_estimators=50)

# Train the classifier on the training data
rf_classifier.fit(X_train, Y_train)

# Make predictions on the test data
predictions = rf_classifier.predict(X_test)

In [ ]:
print(Y_train.isnull().sum())
Y_train = Y_train.fillna(0)
Y_test = Y_test.fillna(0)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy*100)


**Checking Overfitting**

In [ ]:
import random
indices = random.sample(range(len(X_train)), int(len(X_train) * 0.2))
X_train_subset = X_train.iloc[indices]
Y_train_subset = Y_train.iloc[indices]

# Make predictions on the test data
predictions = rf_classifier.predict(X_train_subset)


In [ ]:
print(Y_train.isnull().sum())
Y_train = Y_train.fillna(0)

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(Y_train_subset, predictions)
print("Accuracy:", accuracy*100)

In [ ]:
df["feature_q1_0"].info()

In [ ]:
df.head()

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Define XGBoost classifier and set hyperparameters
xgb_classifier = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8
)

# Train the XGBoost classifier using the training data
xgb_classifier.fit(X_train, Y_train)

# Make predictions on the test data
predictions = xgb_classifier.predict(X_test)

# Calculate accuracy of predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy*100)


Accuracy: 77.605
